This Notebook compares recommendations made by GROQ  based on several prompts.

In [1]:
import pandas as pd

Building the Training Data

In [2]:
train = pd.read_csv('../data/train.csv')

In [3]:
train.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,259,255,4,874724710,My Best Friend's Wedding (1997),Romance,4.0
1,259,286,4,874724727,"English Patient, The (1996)","Romance, War",4.0
2,259,298,4,874724754,Face/Off (1997),"Action, Sci-Fi, Thriller",4.0
3,259,185,4,874724781,Psycho (1960),"Horror, Romance, Thriller",4.0
4,259,173,4,874724843,"Princess Bride, The (1987)","Action, Adventure, Romance",4.0


The movies each user has watched in the training data

In [4]:
user_movies = train.groupby('user_id').apply(lambda x: x['movie_title'].tolist())

C:\Users\Admin\AppData\Local\Temp\ipykernel_21364\3806896237.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_movies = train.groupby('user_id').apply(lambda x: x['movie_title'].tolist())


In [5]:
""" Number of users to run the recommender on
    (Use a small sample for faster results)
    If you want to run the recommender on all users, uncomment the last line
"""
num_users = 20

# Uncomment below to run on all users
# n = user_movies.shape[0]

The most recent movies each user has watched.

In [6]:
def get_user_recent_movies(dataframe, n = 10):
    return dataframe.groupby('user_id').apply(lambda x: x['movie_title'].tail(n).to_list())

user_recent_movies = get_user_recent_movies(train)

C:\Users\Admin\AppData\Local\Temp\ipykernel_21364\117843240.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return dataframe.groupby('user_id').apply(lambda x: x['movie_title'].tail(n).to_list())


The movies each user rated highly.

In [7]:
def get_user_top_movies(dataframe, n = 10):
    return dataframe.groupby('user_id').apply(lambda x: x.sort_values('rating', ascending=False).head(n)['movie_title'].to_list())

user_top_movies = get_user_top_movies(train)

C:\Users\Admin\AppData\Local\Temp\ipykernel_21364\1365838253.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return dataframe.groupby('user_id').apply(lambda x: x.sort_values('rating', ascending=False).head(n)['movie_title'].to_list())


The genres of each movie.

In [8]:
movie_genres = train.groupby('movie_title').apply(lambda x: x['genres'].unique().tolist()).apply(lambda x: x[0].split(", "))

C:\Users\Admin\AppData\Local\Temp\ipykernel_21364\585781067.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  movie_genres = train.groupby('movie_title').apply(lambda x: x['genres'].unique().tolist()).apply(lambda x: x[0].split(", "))


In [11]:
user_top_movies.head(20)

user_id
1     [Empire Strikes Back, The (1980), Usual Suspec...
5     [Wrong Trousers, The (1993), This Is Spinal Ta...
6     [Down by Law (1986), Graduate, The (1967), Lon...
8     [Contact (1997), Die Hard (1988), Braveheart (...
10    [Amadeus (1984), Taxi Driver (1976), All About...
12    [Dead Poets Society (1989), Wizard of Oz, The ...
13    [Wag the Dog (1997), Lawrence of Arabia (1962)...
14    [Great Dictator, The (1940), Stand by Me (1986...
15    [Phenomenon (1996), Nixon (1995), Cry, the Bel...
16    [Aladdin (1992), Fargo (1996), Star Trek IV: T...
17    [Swingers (1996), Liar Liar (1997), Willy Wonk...
18    [Kolya (1996), Local Hero (1983), Babe (1995),...
19    [Butch Cassidy and the Sundance Kid (1969), Ba...
20    [Ghost and the Darkness, The (1996), Searching...
21    [Dead Man Walking (1995), Nosferatu (Nosferatu...
22    [Contact (1997), Fierce Creatures (1997), Youn...
23    [Full Monty, The (1997), Raising Arizona (1987...
24    [Stand by Me (1986), Happy Gilmore

The genres for each users top rated movies

In [36]:
user_top_movie_genres = user_top_movies.apply(lambda x: pd.Series(movie_genres.loc[x].sum()).unique().tolist())

The ratings provided by each user to every movie they watched in the training data.

In [37]:
user_ratings = train.drop_duplicates(['user_id', 'movie_title']).pivot(index='user_id', columns='movie_title', values='avg_rating').fillna(0)

In [38]:
from scipy.sparse import csr_matrix
sparse_user_ratings = csr_matrix(user_ratings)

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Identifying similar users based on cosine similarity

In [40]:
def get_similar_users(sparse_user_ratings, index, n = 10):
    return pd.DataFrame(cosine_similarity(sparse_user_ratings) - np.identity(sparse_user_ratings.shape[0]), index = index, columns = index).apply(lambda x: list(x.sort_values(ascending = False).head(n).index), axis=1)

similar_users = get_similar_users(sparse_user_ratings, user_ratings.index)

In [41]:
similar_users.head()

user_id
1     [916, 457, 268, 435, 429, 823, 301, 276, 889, ...
5      [648, 407, 307, 497, 268, 276, 22, 622, 804, 70]
6       [18, 194, 716, 10, 666, 151, 321, 85, 389, 854]
8      [746, 158, 37, 352, 638, 425, 22, 627, 671, 538]
10      [6, 406, 666, 389, 321, 398, 524, 716, 194, 18]
dtype: object

Identifying Candidate movies which are the movies each user's similar users rated most highly overall.

In [42]:
def get_candidate_movies(similar_users, dataframe, n = 20):
    return similar_users.apply(lambda x: list(dataframe.loc[dataframe['user_id'].isin(x)].groupby('movie_title').sum().sort_values('rating', ascending = False).index[:n]))

candidate_movies = get_candidate_movies(similar_users, train)

In [43]:
user_prompt_data = pd.concat((user_movies, user_top_movies, user_recent_movies, similar_users, candidate_movies, user_top_movie_genres), axis = 1, keys = ['user_movies', 'user_top_movies', 'user_recent_movies', 'similar_users', 'candidate_movies', 'user_top_movie_genres'])
user_prompt_data


,user_movies,user_top_movies,user_recent_movies,similar_users,candidate_movies,user_top_movie_genres
user_id,,,,,,
1,"[Empire Strikes Back, The (1980), Monty Python...","[Empire Strikes Back, The (1980), Usual Suspec...","[Dolores Claiborne (1994), French Twist (Gazon...","[916, 457, 268, 435, 429, 823, 301, 276, 889, ...","[Chasing Amy (1997), Star Wars (1977), Empire ...","[Action, Adventure, Romance, Sci-Fi, War, Thri..."
5,"[unknown, Star Trek: First Contact (1996), Jac...","[Wrong Trousers, The (1993), This Is Spinal Ta...","[Radioland Murders (1994), Houseguest (1994), ...","[648, 407, 307, 497, 268, 276, 22, 622, 804, 70]","[Raiders of the Lost Ark (1981), Monty Python ...","[Animation, Musical, unknown, Action, Adventur..."
6,"[Kolya (1996), English Patient, The (1996), L....","[Down by Law (1986), Graduate, The (1967), Lon...","[Monty Python and the Holy Grail (1974), Bob R...","[18, 194, 716, 10, 666, 151, 321, 85, 389, 854]","[Casablanca (1942), Wizard of Oz, The (1939), ...","[unknown, Romance, Mystery, Adventure, Sci-Fi,..."
8,"[Contact (1997), Liar Liar (1997), In & Out (1...","[Contact (1997), Die Hard (1988), Braveheart (...","[Star Trek: First Contact (1996), Jurassic Par...","[746, 158, 37, 352, 638, 425, 22, 627, 671, 538]","[Blade Runner (1982), Raiders of the Lost Ark ...","[Sci-Fi, Action, Thriller, War, Adventure, unk..."
10,"[Full Monty, The (1997), L.A. Confidential (19...","[Amadeus (1984), Taxi Driver (1976), All About...","[To Wong Foo, Thanks for Everything! Julie New...","[6, 406, 666, 389, 321, 398, 524, 716, 194, 18]","[Casablanca (1942), Wizard of Oz, The (1939), ...","[Mystery, Thriller, unknown, War, Adventure, W..."
...,...,...,...,...,...,...
937,"[Contact (1997), Ulee's Gold (1997), English P...","[Boot, Das (1981), Star Wars (1977), Dead Man ...","[Ridicule (1996), Big Night (1996), Liar Liar ...","[413, 735, 136, 558, 390, 590, 63, 199, 864, 460]","[Fargo (1996), English Patient, The (1996), De...","[Action, War, Adventure, Romance, Sci-Fi, unkn..."
939,"[Jackal, The (1997), Kull the Conqueror (1997)...","[Jackal, The (1997), My Best Friend's Wedding ...","[Set It Off (1996), First Wives Club, The (199...","[718, 935, 238, 357, 181, 141, 759, 891, 15, 159]","[Independence Day (ID4) (1996), Mission: Impos...","[Action, Thriller, Romance, unknown, Adventure..."
940,"[English Patient, The (1996), Everyone Says I ...","[Titanic (1997), Air Force One (1997), Contact...","[Titanic (1997), L.A. Confidential (1997), Mot...","[404, 827, 724, 860, 149, 284, 112, 589, 740, ...","[Air Force One (1997), English Patient, The (1...","[Action, Romance, Thriller, Sci-Fi, unknown, C..."


Collaborative Filtering Prompt

In [44]:
new_line = '\n'

def collabPrompt(each, user_prompt_data, new_line = '\n'):
    return f"""I am user {each}.
The most recent ten movies I have seen are:
{", ".join(user_prompt_data.loc[each, 'user_recent_movies'])}.
My top rated movies are:
{", ".join(user_prompt_data.loc[each, 'user_top_movies'])}.
The users who are most like me are {", ".join([str(each) for each in user_prompt_data.loc[each, 'similar_users']])}.
The top movies for each of these users are:
{new_line.join([f"{each}: {', '.join(user_prompt_data.loc[each, 'user_top_movies'])}" for each in user_prompt_data.loc[each, 'similar_users']])}.
Please recommend ten movies for me to watch that I have not seen. Provide brackets around your recommendations so I can easily parse them.
For example ([Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])"""
    

In [45]:
collab_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: collabPrompt(x, user_prompt_data))

In [46]:
from dotenv import load_dotenv
from groq import Client
import os
dotenv_path = 'D:\\test\\LLM-Recommender-System-with-RAG\\key_api.env'  # Thay thế bằng đường dẫn thực tế
load_dotenv(dotenv_path)
api_key = os.getenv('GROQ_API_KEY')
client = Client(api_key=api_key)

In [47]:
collab_response = collab_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [48]:
collab_recommendations = collab_response.apply(lambda x: x.choices[0].message.content.split("[")[1].split("]")[0].split("\n"))

In [49]:
collab_recommendations.head()

user_id
1     [The Big Sleep (1946) - A classic film noir my...
5     [The Big Sleep (1946), The Philadelphia Story ...
6     [Nashville (1975), The Leopard (1963), The Man...
8                             [The Big Lebowski (1998)]
10    [A Gentleman's Agreement (1947), The Leopard (...
Name: user_id, dtype: object

In [50]:
test = pd.read_csv('../data/test.csv')

In [51]:
user_movies_test = test.groupby('user_id').apply(lambda x: x['movie_title'].tolist())

C:\Users\Admin\AppData\Local\Temp\ipykernel_22144\134710919.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_movies_test = test.groupby('user_id').apply(lambda x: x['movie_title'].tolist())


In [52]:
collab_hits = pd.concat((user_movies, user_movies_test, collab_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'collab_recommendations']).dropna(subset = 'collab_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Prompt Similar to Zero-Shot Paper (reduced to one prompt, added genres)

In [53]:
def genrePrompts(each, user_prompt_data, new_line = '\n'):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?.
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [54]:
genre_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: genrePrompts(x, user_prompt_data))

In [55]:
genre_prompts.head()

user_id
1     \nCandidate Set (candidate movies): ['Chasing ...
5     \nCandidate Set (candidate movies): ['Raiders ...
6     \nCandidate Set (candidate movies): ['Casablan...
8     \nCandidate Set (candidate movies): ['Blade Ru...
10    \nCandidate Set (candidate movies): ['Casablan...
Name: user_id, dtype: object

In [56]:
genre_response = genre_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [57]:
genre_recommendations = genre_response.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [58]:
genre_recommendations.head()

user_id
1     [Silence of the Lambs, The (1991), Pulp Fictio...
5     [Terminator, The (1984), Aliens (1986), Star W...
6     [Star Wars (1977), Godfather, The (1972), M*A*...
8     [Aliens (1986), The Fugitive (1993), Terminato...
10    [Citizen Kane (1941), Bridge on the River Kwai...
Name: user_id, dtype: object

In [59]:
genre_hits = pd.concat((user_movies, user_movies_test, genre_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'genre_recommendations']).dropna(subset = 'genre_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Prompt Similar to Zero-Shot Paper (uses two prompts, adds genres)

In [60]:
def twoStepPrompt1(each, user_prompt_data):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: 
"""

def twoStepPrompt2(each, user_prompt_data, response1):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Their genres are: {user_prompt_data.loc[each, 'user_top_movie_genres']}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: {response1.loc[each]}.
Step 2: Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [61]:
prompt1 = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: twoStepPrompt1(x, user_prompt_data))

In [62]:
response1 = prompt1.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{'role': 'user', 'content': x}],
    max_tokens=512,
    temperature=0.7
).choices[0].message.content.strip())

# Kiểm tra kết quả
response1.head()


user_id
1     Based on the movies you've rated highly, it ap...
5     Based on the movies you have rated highly, it ...
6     Based on the movies you have rated highly, it ...
8     Based on the movies you've rated highly, it ap...
10    Based on the movies you have rated highly, it ...
Name: user_id, dtype: object

In [63]:
response1.head()

user_id
1     Based on the movies you've rated highly, it ap...
5     Based on the movies you have rated highly, it ...
6     Based on the movies you have rated highly, it ...
8     Based on the movies you've rated highly, it ap...
10    Based on the movies you have rated highly, it ...
Name: user_id, dtype: object

In [64]:
prompt2 = pd.Series(user_prompt_data.index, index = user_prompt_data.index).iloc[:num_users].apply(lambda x: twoStepPrompt2(x, user_prompt_data, response1))

In [65]:
response2 = prompt2.apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [66]:
twoStep_recommendations = response2.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [67]:
twoStep_recommendations.head()

user_id
1     [Schindler's List (1993), Raiders of the Lost ...
5     [Monty Python and the Holy Grail (1974), Alien...
6     [, Philadelphia Story, The (1940), Monty Pytho...
8     [Empire Strikes Back, The (1980), Terminator, ...
10    [The African Queen (1951), The Bridge on the R...
Name: user_id, dtype: object

In [68]:
twoStep_hits = pd.concat((user_movies, user_movies_test, twoStep_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'twoStep_recommendations']).dropna(subset = 'twoStep_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

In [69]:
twoStep_hits.head()

user_id
1     9
5     8
6     7
8     6
10    5
dtype: int64

Prompt using wikipedia movie summaries

In [70]:
movie_wiki = pd.read_csv('../data/movie_wiki.csv')
movie_wiki.head()

,movie_title,wiki_summary
0,'Til There Was You (1997),'Til There Was You is a 1997 American romantic...
1,1-900 (1994),1-900 or 06 is a 1994 Dutch erotic romantic dr...
2,101 Dalmatians (1996),101 Dalmatians is a 1996 American adventure co...
3,12 Angry Men (1957),12 Angry Men is a 1957 American independent le...
4,187 (1997),One Eight Seven (also known as 187) is a 1997 ...


In [71]:
def wikiPrompt(each, user_prompt_data, movie_wiki, new_line = '\n'):
    return f"""
Candidate Set (candidate movies): {user_prompt_data.loc[each, 'candidate_movies']}.
The movies I have rated highly (watched movies): {user_prompt_data.loc[each, 'user_top_movies']}.
Summary of the movies I have watched: {new_line.join([f"{eachMovie}: {movie_wiki.loc[movie_wiki['movie_title'] == eachMovie, 'wiki_summary'].iloc[0]}" for eachMovie in user_prompt_data.loc[1, 'user_top_movies']])}
Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?.
Please use brackets around the movies you recommend and separate the titles by new lines so I can easily parse them.
(Format Example: Here are the 10 movies recommended for you: [Midnight Cowboy (1969){new_line}Lost in Translation (2003){new_line}etc.])
Answer: 
"""

In [72]:
wiki_prompts = pd.Series(user_prompt_data.index, index = user_prompt_data.index).apply(lambda x: wikiPrompt(x, user_prompt_data, movie_wiki))

In [73]:
wiki_prompts.head()

user_id
1     \nCandidate Set (candidate movies): ['Chasing ...
5     \nCandidate Set (candidate movies): ['Raiders ...
6     \nCandidate Set (candidate movies): ['Casablan...
8     \nCandidate Set (candidate movies): ['Blade Ru...
10    \nCandidate Set (candidate movies): ['Casablan...
Name: user_id, dtype: object

In [74]:
wiki_response = wiki_prompts.iloc[:num_users].apply(lambda x: client.chat.completions.create(
    model="llama3-8b-8192",
        messages= [{ 'role':'user','content' : x}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [75]:
wiki_recommendations = wiki_response.apply(lambda x: x.choices[0].message.content.split("[")[-1].split("]")[0].split("\n"))

In [76]:
wiki_recommendations.head()

user_id
1     [Chasing Amy (1997), Raiders of the Lost Ark (...
5     [Monty Python and the Holy Grail (1974), Empir...
6     [Amadeus (1984), Philadelphia Story, The (1940...
8     [Rock, The (1996), Jurassic Park (1993), Indep...
10    [Casablanca (1942), Butch Cassidy and the Sund...
Name: user_id, dtype: object

In [84]:
wiki_hits = pd.concat((user_movies, user_movies_test, wiki_recommendations), axis=1, keys = ['user_movies', 'user_movies_test', 'wiki_recommendations']).dropna(subset = 'wiki_recommendations').apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(x.iloc[2])) if x.iloc[1] is not np.nan else len(set(x.iloc[0]).intersection(x.iloc[2])), axis = 1)

Baseline Recommender: Recommend top 10 most popular movies to every user

In [85]:
viewings = train.groupby('movie_title').count().sort_values('user_id', ascending=False)

In [86]:
viewings.head()

,user_id,movie_id,rating,timestamp,genres,avg_rating
movie_title,,,,,,
Star Wars (1977),393,393,393,393,393,393
Fargo (1996),355,355,355,355,355,355
Return of the Jedi (1983),350,350,350,350,350,350
Liar Liar (1997),317,317,317,317,317,317
Toy Story (1995),315,315,315,315,315,315


In [87]:
top_10_movies = viewings.head(10).index.tolist()
top_10_movies

['Star Wars (1977)',
 'Fargo (1996)',
 'Return of the Jedi (1983)',
 'Liar Liar (1997)',
 'Toy Story (1995)',
 'English Patient, The (1996)',
 'Independence Day (ID4) (1996)',
 'Contact (1997)',
 'Scream (1996)',
 'Raiders of the Lost Ark (1981)']

In [81]:
baseline_hits = pd.concat((user_movies, user_movies_test), axis=1).dropna().iloc[:num_users].apply(lambda x: len(set(x.iloc[0]).union(x.iloc[1]).intersection(top_10_movies)), axis = 1)

Comparison of Prompts

In [82]:
comparison = pd.concat((collab_hits, genre_hits, twoStep_hits, wiki_hits, baseline_hits), axis = 1, keys = ['collab_hits', 'genre_hits', 'twoStep_hits', 'wiki_hits', 'baseline_hits'])

Below values are Hit Rate in % (i.e. 100 is 100%)

In [83]:
comparison.describe().iloc[1:] / 10 * 100

,collab_hits,genre_hits,twoStep_hits,wiki_hits,baseline_hits
mean,1.500000,63.500000,52.500000,65.000000,65.000000
std,3.663475,27.772573,27.696665,31.705885,20.900768
min,0.000000,0.000000,0.000000,10.000000,20.000000
25%,0.000000,50.000000,30.000000,55.000000,60.000000
50%,0.000000,70.000000,55.000000,75.000000,70.000000
75%,0.000000,82.500000,72.500000,90.000000,72.500000
max,10.000000,90.000000,90.000000,100.000000,100.000000
